In [94]:
import numpy as np
from simplifiedLikelihoods import Data,UpperLimitComputer
import pandas as pd

In [106]:
metBins = [250,  280,  310,  340,  370,  400,  430,  470,  510, 550,  590,  640,  690,  
            740,  790,  840,  900,  960, 1020, 1090, 1160, 1250, 1400]

In [95]:
cmsYieldFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:Yields(Monojet).csv'
bgYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=5,
                          skip_footer=67,names=True,dtype=None,encoding=None)

dataYields = np.genfromtxt(cmsYieldFile,delimiter=',',skip_header=73,
                           names=True,dtype=None,encoding=None)

covarianceFile = '../AuxInfo/CMS-EXO-20-004-data/HEPData-ins1894408-v2-csv/Simplifiedlikelihood:covariancematrix(Monojet).csv'
cov = np.genfromtxt(covarianceFile,delimiter=',',skip_header=5,
                           names=True,dtype=None,encoding='UTF-8')

In [97]:
srOrder = cov['Second_Bin'][:66]
covMatrix = np.zeros((len(srOrder),len(srOrder)))
for pt in cov:
    i = np.where(srOrder == pt['First_Bin'])
    j = np.where(srOrder == pt['Second_Bin'])
    covMatrix[i,j] = pt['Covariance']

In [108]:
nobs = [int(dataYields[dataYields['Bin'] == sr]['Data_yield'][0]) for sr in srOrder]
nbg = [float(bgYields[bgYields['Bin'] == sr]['Background_yield'][0]) for sr in srOrder]

In [115]:
f = '../axial_2000_1.pcl'
recastData = pd.read_pickle(f)
ns = []
for sr in srOrder:
    dataset,ibin = sr.split('_')[1:]
    dataset = eval(dataset)
    ibin = eval(ibin.replace('bin',''))
    binLabel = 'bin_%1.1f_%1.1f' %(metBins[ibin],metBins[ibin+1])
    signalYield = recastData[recastData['Data-takingperiod'] == dataset][binLabel].iloc[0]
    ns.append(signalYield)
ns = np.array(ns)

In [132]:
data = Data(observed=nobs, backgrounds=nbg, covariance=covMatrix, 
            nsignal=1.*ns,deltas_rel=0.0)
dataExp = Data(observed=[int(b) for b in nbg], backgrounds=nbg, covariance=covMatrix, 
               nsignal=1.*ns,deltas_rel=0.0)

In [133]:
ulComp = UpperLimitComputer()

In [134]:
ul = ulComp.getUpperLimitOnMu(data)
ulExp = ulComp.getUpperLimitOnMu(dataExp)

In [135]:
print(ul,ulExp)

1.5740807808972614 0.7337937439181763
